In [3]:
import autograd.numpy as np
from autograd import grad
from autograd import hessian
import scipy.optimize as opt
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

## 2-nests. Nest A and Nest B both have two products, respectively. Namely, $\{1,2\} \in A$, and $\{3,4\} \in B$

## Data Generating Process

- Utility function

$$
U_{nj}  = -\beta_1 \frac{price_j}{income_{n}}+ \beta_2 quality_{j} + \epsilon_{nj}
$$
- Four alternatives
- Two nests in which there are two alternatives
- 100 consumers
- $\epsilon_{nj}$ follows GEV distribution with parameters $<\lambda_1, \lambda_2>$
- $\lambda_1 = 1.2$
- $\lambda_2 = 1.5$

In [1]:
k1 = 2
k2 = 2
K = 2

N = 100


b1 = -1.4
b2 = .8
lda1 = 1.2
lda2 = 1.5


# User-level characteristics
true_params = np.array([b1,b2,lda1,lda2])
income = np.random.uniform(low = 2,high = 5, size = N)
uid = np.arange(N)
user = pd.DataFrame(data = np.concatenate([uid.reshape(-1,1),income.reshape(-1,1)],axis=1),columns = ['uid','income'])

# Product-level characteristics
price = abs(np.random.normal(scale = 2,loc=1.3, size = k1 + k2).reshape(-1,1))
quality = abs(np.random.normal(scale = 2,loc =3.2, size = k1 + k2).reshape(-1,1))

gid = np.arange(k1+k2)
goods = pd.DataFrame(data = np.concatenate([gid.reshape(-1,1),price, quality],axis=1),columns = ['gid','price','quality'])


# Merged dataset, shape = (N * K,2)
data = user.merge(goods, how = 'cross')
data['nest'] = 0
data.loc[data.query('gid<2').index,'nest'] = 1
data.loc[data.query('gid>=2').index,'nest'] = 2
data['p/i'] = data['price']/data['income']
dataset = data

NameError: name 'np' is not defined

In [2748]:
def generate_sample(params):
    """
    Params: params---> all paramters drew by certain criterions or randomly.
    Return: P_j ---> predicted probability for each choice for each consumer.
    Description: This function uses for generating true choice probability under true parameters
    """
    b = params[:2]
    lda = params[2:]
    V = X.reshape(N,4,2).dot(b.reshape(-1,1))
    V = V - V.mean(axis=1).reshape(N,1,1)
    w = V.reshape(N,2,2).mean(axis=2)
    y = V.reshape(N,2,2) - w.reshape(N,2,1)
    Ink = np.log(np.exp(y/(lda.reshape(-1,1))).sum(axis=2))
    Ink_lda = Ink * lda
    Pk = np.exp(w + Ink_lda)
    Pk = Pk/Pk.sum(axis=1,keepdims=True)
    P_j_given_k = np.exp(V.reshape(N,2,2)/(lda.reshape(-1,1)))
    P_j_given_k = P_j_given_k/P_j_given_k.sum(axis=2,keepdims=True)
    P_j = Pk.reshape(N,-1,1) * P_j_given_k 
    return P_j

In [2749]:
def get_choice(p_draw):
    """
    Params: p_draw ----> drawing probabilities
    Return: predicted choice under p_draw
    Description: This function predicts each consumer's choice when using true parameters
    """
    true_choice = np.zeros(N)
    for i in range(len(true_choice)):
        true_choice[i] = (np.random.choice((dataset['gid'].values.reshape(N,-1))[i],p = (p_draw.reshape(N,-1))[i]))
    true_choice = true_choice.astype(int)
    choice_set = np.zeros(shape=(N,4))
    for i in range(len(true_choice)):
        choice_set[i, true_choice[i]] = 1
    return choice_set


In [2750]:

dataset = dataset.sort_values(by = ['uid','gid'])

X = np.array(dataset[['p/i','quality']].values)
p_draw =  generate_sample(true_params)
observed_choice = get_choice(p_draw)


In [2751]:
def lln(params):
    '''
    Params: params ----> all parameters in the model
    Return: Log-likelihood function
    Description: This function calculates the log-likelihood under a set of parameters using sequential method (conditional probability)
    '''
#     params = np.exp(params)
    b = params[:2]
    lda = params[2:]
    V = X.reshape(N,4,2).dot(b.reshape(-1,1))
    V = V - V.mean(axis=1).reshape(N,1,1)
    w = V.reshape(N,2,2).mean(axis=2)
    y = V.reshape(N,2,2) - w.reshape(N,2,1)
    Ink = np.log(np.exp(y/(lda.reshape(-1,1))).sum(axis=2))
    Ink_lda = Ink * lda
    Pk = np.exp(w + Ink_lda)
    Pk = Pk/Pk.sum(axis=1,keepdims=True)
    P_j_given_k = np.exp(V.reshape(N,2,2)/(lda.reshape(-1,1)))
    P_j_given_k = P_j_given_k/P_j_given_k.sum(axis=2,keepdims=True)
    P_j = Pk.reshape(N,-1,1) * P_j_given_k
    
    return -np.sum(np.log(P_j).reshape(N,-1)* observed_choice)

In [2752]:
def get_ll(params):
        '''
    Params: params ----> all parameters in the model
    Return: Log-likelihood function
    Description: This function calculates the log-likelihood under a set of parameters using MLE
    '''
    b = params[:2]
    lda = params[2:]
    V = X.reshape(N,4,2).dot(b.reshape(-1,1))
    V = V - V.mean(axis=1).reshape(N,1,1)
    e_p = np.exp(V.reshape(N,2,2)/(lda.reshape(-1,1)))
    ep_sum = e_p.sum(axis=2)
    P_j = e_p * (ep_sum**(lda-1)).reshape(N,2,1)/(((ep_sum**(lda)).sum(axis=1)).reshape(N,1,1)) 
    return -np.sum(np.log(P_j).reshape(N,-1)* observed_choice)

In [2753]:
#### Verifying our data
print('True Log-likelihood using sequential method:-', lln(true_params))
print('True Log-likelihood using MLE method:-', get_ll(true_params))
print('\n Therefore, two methods result in the same log-likelihood at the true parameters')

True Log-likelihood using sequential method:- 113.79051134754229
True Log-likelihood using MLE method:- 113.79051134754229

 Therefore, two methods result in the same log-likelihood at the true parameters


## In this initial values, both methods converge to the true solution

In [2778]:
params = np.array([0.5,0.9,0.1,.1])
res = opt.minimize(lln,x0=params,method = 'L-BFGS-B')
print('---------------------')
print('Sequential method estimates:')
print('Estiamted log-likelihood = ', -res.fun)
print('Initial parameters = ', params)
print('Predicted parameters = ', res.x)
print('True parameters = ', true_params)



res = opt.minimize(get_ll,x0=params,method = 'L-BFGS-B')
print('---------------------')
print('MLE estimates:')
print('Estiamted log-likelihood = ', -res.fun)
print('Initial parameters = ', params)
print('Predicted parameters = ', res.x)
print('True parameters = ', true_params)




---------------------
Sequential method estimates:
Estiamted log-likelihood =  -113.49761668287528
Initial parameters =  [0.5 0.9 0.1 0.1]
Predicted parameters =  [-1.39119947  1.02120133  1.3017994   1.49873968]
True parameters =  [-1.4  0.8  1.2  1.5]
---------------------
MLE estimates:
Estiamted log-likelihood =  -113.4976166828102
Initial parameters =  [0.5 0.9 0.1 0.1]
Predicted parameters =  [-1.39119751  1.02119958  1.3017973   1.4987374 ]
True parameters =  [-1.4  0.8  1.2  1.5]


## However, under this initial vector, Sequential method converges to the true value while the MLE converges to a local min

In [2821]:
params = np.random.normal(size=4,scale=2)

res = opt.minimize(lln,x0=params,method = 'L-BFGS-B')
print('---------------------')
print('Sequential method estimates:')
print('Estiamted log-likelihood = ', -res.fun)
print('Initial parameters = ', params)
print('Predicted parameters = ', res.x)
print('True parameters = ', true_params)



res = opt.minimize(get_ll,x0=params,method = 'L-BFGS-B')
print('---------------------')
print('MLE estimates:')
print('Estiamted log-likelihood = ', -res.fun)
print('Initial parameters = ', params)
print('Predicted parameters = ', res.x)
print('True parameters = ', true_params)




---------------------
Sequential method estimates:
Estiamted log-likelihood =  -113.49761668210584
Initial parameters =  [-2.55289528  2.63527582 -5.036244   -0.59738734]
Predicted parameters =  [-1.39117254  1.0211664   1.30175949  1.49871583]
True parameters =  [-1.4  0.8  1.2  1.5]
---------------------
MLE estimates:
Estiamted log-likelihood =  -114.21229265760132
Initial parameters =  [-2.55289528  2.63527582 -5.036244   -0.59738734]
Predicted parameters =  [-0.01541516 -0.23023235 -0.16775621 -0.07156722]
True parameters =  [-1.4  0.8  1.2  1.5]


# Archived
## Archived data generating process
Here I comment out another method of DGP which I ever used

In [ ]:
# DGP 1

# k1 = 2
# k2 = 2
# K = 2
# ## 2000 consumers
# N = 2000


# b1 = -1.4
# b2 = .8
# lda1 = 1.2
# lda2 = 1.5
# params = np.array([b1,b2,lda1,lda2])
# true_params = np.array([b1,b2,lda1,lda2])

# income = np.random.uniform(low = 1,high = 3, size = N)
# uid = np.arange(N)
# user = pd.DataFrame(data = np.concatenate([uid.reshape(-1,1),income.reshape(-1,1)],axis=1),columns = ['uid','income'])


# price = abs(np.random.normal(scale = 2,loc=1.3, size = k1 + k2).reshape(-1,1))
# quality = abs(np.random.normal(scale = 2,loc =3.2, size = k1 + k2).reshape(-1,1))
# gid = np.arange(k1+k2)


# goods = pd.DataFrame(data = np.concatenate([gid.reshape(-1,1),price, quality],axis=1),columns = ['gid','price','quality'])


# data = user.merge(goods, how = 'cross')
# data['nest'] = 0
# data.loc[data.query('gid<2').index,'nest'] = 1
# data.loc[data.query('gid>=2').index,'nest'] = 2
# data['p/i'] = data['price']/data['income']
# data['v'] = data[['p/i','quality']].dot(params[:2])
# data['w'] =  data.groupby(['nest','uid'])['v'].transform('mean')
# data['y'] = data['v'] - data['w']
# data['lda1'] = lda1
# data['lda2'] = lda2
# data['lda'] = 0.
# data.loc[data.query('nest == 1').index,'lda'] = data['lda1']
# data.loc[data.query('nest == 2').index,'lda'] = data['lda2']

# def calc_i(df):
#     return np.log(np.exp(df['y']/df['lda']).sum())

# Ink = data.groupby(['nest','uid']).apply(calc_i).reset_index().rename({0:'I'},axis=1)
# data = data.merge(Ink)
# data['lda_i'] = data['lda'] * data['I']

# ee1 = np.random.gumbel(scale = lda1, size = N * k1)
# ee2 = np.random.gumbel(scale = lda2, size = N * k2)
# dataset = data.sort_values(by = ['nest','uid'])
# dataset['ee'] = np.concatenate([ee1,ee2])
# e = np.random.gumbel(scale = 1, size = N*K)
# ek = pd.DataFrame(np.concatenate([np.repeat(uid,2).reshape(-1,1),np.array([0.,1.]*N).reshape(-1,1),e.reshape(-1,1)],axis=1),columns = ['uid','nest','e'])
# ek = ek.sort_values(by = ['nest','uid'])
# dataset['e'] = np.concatenate([np.repeat(ek['e'][:N],2), np.repeat(ek['e'][N:],2)]) 
# dataset['xi']=0
# dataset[dataset['nest'] == 1].loc[:,'xi'] = np.random.gumbel(0,1)
# dataset[dataset['nest'] == 2].loc[:,'xi'] = np.random.gumbel(0,1)
# dataset['u'] = dataset['y'] + dataset['w'] +dataset['lda_i'] + dataset['e'] + dataset['xi']
# dataset['max_u'] = dataset.groupby(['uid'])['u'].transform(max)
# outcome = dataset.query('(max_u == u)' )[['uid','gid']].rename({'gid':'outcome'},axis=1)
# dataset = dataset.merge(outcome)

# X = np.array(dataset[['p/i','quality']].values)
# dataset['choice'] = 0
# dataset.loc[dataset.query('outcome == gid').index,'choice']=1
# choice = np.array(dataset['choice'])